# Load Libraries :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage import gaussian_filter, sobel

# 1 - Fundamental Level Sets : 

## Introduction 
Instead of representing the active contour explicitly by a set of ordered point, We define the contour implicitely by the zero level set of a function phi(x,y), by evolving this function in time we can approch our desired contour.  
During evolution:
- *ϕ* is updated over time using a **partial differential equation (PDE)**.
- The idea is that *ϕ* deforms under some speed function so that the zero-level set (the contour) moves toward the object boundary in the image.

## Mathematical Formulation

The basic level set method represents a contour 𝐶 implicitly as the zero level set of a higher-dimensional function ϕ(x,y,t), where:
- (x,y) are spatial coordinates
- t is time/iteration
- The contour C is defined where ϕ(x,y,t) = 0

The contour evolution is governed by the level set equation:

$$\frac{\partial \phi}{\partial t} + F|\nabla \phi| = 0$$

Where:
- F is the speed function that controls the contour evolution
- ∇ϕ is the spatial gradient of ϕ

## Basic Algorithm Components

### 1. Level Set Function Initialization

Typically, the level set function is initialized as a signed distance function:
- ϕ(x,y,0) < 0 for points inside the initial contour
- ϕ(x,y,0) > 0 for points outside
- |ϕ(x,y,0)| represents the distance to the contour

A common initialization is:
$$\phi(x,y,0) = \pm d$$

Where d is the distance from (x,y) to the initial contour, and the sign depends on whether the point is inside (negative) or outside (positive).

### 2. Speed Function Design

For image segmentation, the speed function F typically depends on:
- Image gradient: to stop the contour at edges
- Curvature: to maintain smoothness

A basic speed function:
$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$

Where:
- g is an edge detector function (e.g., g = 1/(1+|∇I|²))
- I is the image
- κ is the curvature of the level set
- c (optional) is a constant (balloon force)

### 3. Level Set Evolution

At each iteration, update ϕ using:
$$\phi^{n+1} = \phi^n - \Delta t \cdot F|\nabla \phi^n|$$


#### Time Step Selection

For stability, the time step must satisfy the CFL condition:
$$\Delta t \leq \frac{\Delta x}{max(|F|)}$$

## Complete Algorithm

```
1. Initialize φ(x,y,0) as a signed distance function
2. Calculate the edge indicator function g(|∇I|)
3. For each iteration n:
   a. Calculate ∇φⁿ using finite differences
   b. Compute curvature κ
   c. Evaluate speed function F
   d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
   e. Periodically reinitialize φ to maintain signed distance property
4. Extract final contour from zero level set φ = 0
```



## Implementation :

In [ ]:
# Load image
img = cv2.imread("coins.jpg", cv2.IMREAD_GRAYSCALE)

plt.figure()
plt.imshow(img, cmap='gray')
plt.title('Input Image')
plt.axis('off')
plt.show()

### 1. Initialize the LSF : φ(x,y,0)

In [ ]:
# 1. Initialize the LSF : φ(x,y,0)
def initialize_level_set_circle(shape, center=None, radius=None):
    """
    Initializes a level set function phi as a signed distance function to a circle.

    Parameters:
    - shape: tuple of ints, (height, width) of the domain
    - center: tuple (cx, cy) for the center of the circle; defaults to image center
    - radius: int, radius of the initial circle

    Returns:
    - phi: 2D numpy array of shape `shape`, representing the level set function
    """
    h, w = shape
    if center is None:
        center = (w // 2, h // 2)

    if radius is None:
        radius = (h // 2) - (h // 20)

    x = np.arange(0, w)
    y = np.arange(0, h)
    X, Y = np.meshgrid(x, y)

    # Signed distance to the circle: negative inside, zero on boundary, positive outside
    phi_init = np.sqrt((X - center[0])**2 + (Y - center[1])**2) - radius

    # Normalize the final speed function
    phi_init = phi_init / (np.max(np.abs(phi_init)) + 1e-10)  # Normalize to [-1, 1]

    return phi_init

phi_init = initialize_level_set_circle(img.shape)

plt.figure()
plt.imshow(phi_init, cmap='coolwarm')
plt.colorbar()
plt.contour(phi_init, levels=[0], colors='r')  # zero level set
plt.title('Initial LSF')
plt.axis('off')
plt.show()

### 2. Calculate the edge indicator function g(|∇I|)
$$ g(|\nabla I|) = \frac{1}{1 + |\nabla I|^2} $$  
This function is **small near strong edges**, causing the evolution to slow down and stop there

In [ ]:
# 2. Calculate the edge indicator function g(|∇I|)
def compute_edge_indicator(image, sigma=None):
    """
    Compute edge indicator function: g = 1 / (1 + |∇I|^2)

    Parameters:
    - image: 2D array (grayscale)
    - sigma: float, standard deviation for Gaussian smoothing

    Returns:
    - g: 2D array, same shape as image
    """

    if sigma is None:
        sigma=(img.shape[1]/50)

    # Smooth the image to suppress noise
    smoothed = gaussian_filter(image, sigma=sigma)
    
    # Compute image gradients
    Ix = sobel(smoothed, axis=0)
    Iy = sobel(smoothed, axis=1)
    
    # Gradient magnitude squared
    grad_mag_sq = Ix**2 + Iy**2
    
    # Edge indicator function
    g = 1.0 / (1.0 + grad_mag_sq)

    g = gaussian_filter(g, sigma=sigma)
    
    return g

g = compute_edge_indicator(img)

plt.figure()
plt.imshow(g, cmap='gray')
plt.colorbar()
plt.title('edge indicator function g(|∇I|)')
plt.axis('off')
plt.show()

### 3. For each iteration n:
####    a. Calculate ∇φⁿ using finite differences  
- phi_x, phi_y for directional updates and geometric flow
- |∇φ| for evolving the zero level set in the normal direction

In [ ]:
# 3. For each iteration n:
#    a. Calculate ∇φⁿ using finite differences
def compute_phi_gradient(phi):
    """
    Compute the gradient of the level set function φ.

    Parameters:
    - phi: 2D array, the level set function

    Returns:
    - phi_x: derivative of φ in x direction (axis=1, columns)
    - phi_y: derivative of φ in y direction (axis=0, rows)
    - grad_phi_mag: gradient magnitude sqrt(φ_x^2 + φ_y^2)
    """
    grad_phi_y, grad_phi_x = np.gradient(phi)  # order: rows (y), columns (x)
    grad_phi_mag = np.sqrt(grad_phi_x**2 + grad_phi_y**2) + 1e-10  # small epsilon to avoid division by 0
    return grad_phi_x, grad_phi_y, grad_phi_mag
    
grad_phi_x, grad_phi_y, grad_phi_mag = compute_phi_gradient(phi_init)

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(grad_phi_x, cmap='gray')
plt.colorbar()
plt.title('grad_phi_x')

plt.subplot(1,3,2)
plt.imshow(grad_phi_y, cmap='gray')
plt.colorbar()
plt.title('grad_phi_y')

plt.subplot(1,3,3)
plt.imshow(grad_phi_mag, cmap='gray')
plt.colorbar()
plt.title('grad_phi_mag')

plt.tight_layout()
plt.show()

####    b. Compute curvature κ  

$$ \kappa = \nabla \cdot \left( \frac{\nabla \phi}{|\nabla \phi|} \right) $$  
This curvature term is useful for:
- **Smoothing** the contour
- Regularizing its shape during evolution

In [ ]:
def compute_curvature(phi):
    """
    Compute curvature κ of the level set function φ.

    Parameters:
    - phi: 2D array, the level set function

    Returns:
    - curvature: 2D array of curvature values
    """
    # First derivatives
    phi_y, phi_x = np.gradient(phi)
    norm = np.sqrt(phi_x**2 + phi_y**2) + 1e-10  # avoid division by 0

    # Normalized gradients
    n_x = phi_x / norm
    n_y = phi_y / norm

    # Second derivatives
    nxx, _ = np.gradient(n_x)
    _, nyy = np.gradient(n_y)

    # Divergence of normalized gradient = curvature
    curvature = nxx + nyy
    return curvature

curvature = compute_curvature(phi_init)

plt.figure()
plt.imshow(curvature, cmap='coolwarm')
plt.colorbar()
plt.title('curvature')
plt.axis('off')
plt.show()



#### c. Evaluate speed function F  

$$ F = g \cdot \kappa + \nabla g \cdot \nabla \phi $$
This form is equivalent to:
$$ F = g \cdot \kappa + g_x \cdot \phi_x + g_y \cdot \phi_y $$

In [ ]:
def compute_speed_function(phi, g):
    """
    Compute speed function F = g * κ + ∇g · ∇φ with normalization.

    Parameters:
    - phi: 2D array, level set function
    - g: 2D array, edge indicator function

    Returns:
    - F: 2D array, normalized speed function at each pixel
    """
    # Compute gradients
    phi_y, phi_x = np.gradient(phi)
    g_y, g_x = np.gradient(g)
    
    # Gradient magnitude
    grad_phi_mag = np.sqrt(phi_x**2 + phi_y**2 + 1e-10)  # Avoid division by zero
    grad_phi_mag = grad_phi_mag / np.max(grad_phi_mag)  # Normalize to [0, 1]

    # Curvature
    kappa = compute_curvature(phi)
    kappa = np.clip(kappa, -1, 1)  # Normalize by clamping to [-1, 1]

    # Dot product ∇g · ∇φ
    dot_product = g_x * phi_x + g_y * phi_y
    dot_product = dot_product / (np.max(np.abs(dot_product)) + 1e-10)  # Normalize to [-1, 1]

    # Speed function
    speed_function = g * kappa + dot_product
    
    # Normalize the final speed function
    speed_function = speed_function / (np.max(np.abs(speed_function)) + 1e-10)  # Normalize to [-1, 1]
    
    return speed_function

speed_function = compute_speed_function(phi_init, g)

plt.figure()
plt.imshow(speed_function, cmap='coolwarm')
plt.colorbar()
plt.title('Speed Function')
plt.axis('off')
plt.show()

#### d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
$$ \phi^{t+1} = \phi^t + \Delta t \cdot F |\nabla \phi| $$

In [ ]:
phi_n = phi_init
time_step = 1

phi_n = phi_n + time_step * speed_function * grad_phi_mag

#### e. Run iterative loop

In [ ]:
def level_set_contour_detection(image, nb_iter, center_initial=None, radius_initial=None, time_step=1.0):
    """
    Perform level set evolution for contour detection.
    
    Parameters:
    - image: 2D array, grayscale image
    - nb_iter: int, number of iterations
    - center_initial: tuple, (x, y) coordinates of initial circle center
    - radius_initial: float, radius of initial circle
    - time_step: float, time step Δt
    
    Returns:
    - phi_n: 2D array, final level set function
    """
    # Initialize the LSF
    phi_init = initialize_level_set_circle(image.shape, center_initial, radius_initial)
    
    # Edge indicator function
    g = compute_edge_indicator(image)
    
    # Display inputs
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_init, levels=[0], colors='r')
    plt.title('Input Image and Initial LSF Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(g, cmap='gray')
    plt.title('Edge Indicator Function')
    plt.colorbar()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Initialize the loop
    phi_n = phi_init.copy()
    
    for i in range(nb_iter):
        # Compute ∇φ and |∇φ|
        phi_x, phi_y, grad_phi_mag = compute_phi_gradient(phi_n)
        
        # Compute speed function
        speed_function = compute_speed_function(phi_n, g)
        
        # Update φ
        phi_n = phi_n + time_step * speed_function * grad_phi_mag
        
        # Display every 10 iterations
        if (i % 10) == 0 :
            plt.figure(figsize=(15, 5))

            plt.subplot(1, 3, 1)
            plt.imshow(image, cmap='gray')
            plt.contour(phi_n, levels=[0], colors='r')
            plt.title('iteration Contour')
            plt.axis('off')
            
            plt.subplot(1, 3, 2)
            plt.imshow(speed_function, cmap='coolwarm')
            plt.colorbar()
            plt.title('Speed Function')
            
            plt.subplot(1, 3, 3)
            plt.imshow(phi_n, cmap='coolwarm')
            plt.colorbar()
            plt.title('Level Set Function')
            
            plt.suptitle(f"Iteration {i+1}")
            plt.tight_layout()
            plt.show()
    
    # Final display
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('Final Contour')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(phi_n, cmap='coolwarm')
    plt.colorbar()
    plt.contour(phi_n, levels=[0], colors='r')
    plt.title('Final LSF')
    plt.axis('off')
    
    plt.suptitle("Final Display")
    plt.tight_layout()
    plt.show()
    
    return phi_n

In [ ]:
img = cv2.imread("coins.jpg", cv2.IMREAD_GRAYSCALE)

phi_n = level_set_contour_detection(img, 200)

In [ ]:

#    c. Evaluate speed function F
#    d. Update φⁿ⁺¹ = φⁿ - Δt·F|∇φⁿ|
#    e. Periodically reinitialize φ to maintain signed distance property
# 4. Extract final contour from zero level set φ = 0

### 6. Reinitialization

Over time, φ may lose its signed distance property. Reinitialization solves:
$$\frac{\partial \phi}{\partial \tau} = sign(\phi_0)(1-|\nabla \phi|)$$

Until steady state, where φ₀ is the level set function before reinitialization.

Would you like me to explain any specific part of the algorithm in more detail or demonstrate how it would be implemented in code?